In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings, itertools
from contextlib import contextmanager
from functools import total_ordering
from abc import ABC, abstractmethod
from collections.abc import Sequence
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.dpi': 150, 'font.size': 12, 'axes.titlesize': 'large',
    'axes.labelsize': 'medium', 'xtick.labelsize': 'small', 'ytick.labelsize': 'small'
})
np.set_printoptions(suppress=True, precision=4, linewidth=120)

# --- Utility Functions ---
def note(msg, **kwargs):
    """Prints a formatted message with a notebook icon."""
    formatted_msg = textwrap.fill(msg, width=100, subsequent_indent='   ')
    print(f"\n📝 {formatted_msg}", **kwargs)

def sec(title):
    """Prints a formatted section title for code blocks."""
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 1: Foundations
## Chapter 1.4: The Python Data Model: The Grammar of Pythonic Objects

### Introduction: Why Economists Should Care About Python's 'Grammar'

Python's distinctive character—often described as "Pythonic"—is not an accident of syntax but the result of a deliberate design philosophy centered on consistency and readability. This philosophy is implemented through the **Python data model**, a formal specification that acts as a bridge between the language's core syntax and the objects you create and use. It is, in essence, the grammar of Python objects.

The data model provides a set of protocols that allow user-defined objects to integrate seamlessly with the language's most natural idioms. It is the reason `len(my_collection)` is the universal way to get the size of any collection, from a built-in `list` to a complex `pandas.DataFrame`, rather than a class-specific method like `my_collection.length()`. It is the mechanism that allows the `+` operator to signify integer addition, string concatenation, and vector addition for a NumPy array. This is achieved by mapping the language's syntax to a set of special methods, often called "dunder" (double-underscore) methods, that you implement in your classes.

When the Python interpreter encounters an operation like `len(obj)` or `obj1 + obj2`, it does not look for a method named `len` or `add`. Instead, it translates the operation into a call to a special method on the object itself: `obj.__len__()` or `obj1.__add__(obj2)`. By implementing these methods, you are effectively teaching Python's grammar to your custom objects.

For a computational economist, mastering the data model is a practical necessity. It is the key to creating powerful, domain-specific types—a `PolicyVector`, a `ValueFunction`, a `TransitionMatrix`—that are as intuitive, robust, and easy to use as Python's built-in types. This leads to code that is more readable, less error-prone, and ultimately more scientifically credible. It is the difference between writing a script that works and engineering a tool that is a pleasure to use.

### 1. Code Lab: A Pythonic `Vector` Class

To provide a concrete demonstration, we will construct a `Vector` class that correctly implements several key data model protocols. This exercise will show how a few special methods can transform a basic class into an object that feels native to the language.

We will implement protocols for:
- **Representation:** `__repr__`, `__str__`
- **Equality and Hashing:** `__eq__`, `__hash__` (and the critical link between them)
- **Container Emulation:** `__len__`, `__getitem__`
- **Numeric Operations:** `__add__`, `__mul__`, `__rmul__`, `__abs__`, `__matmul__`
- **Boolean Content:** `__bool__`

In [ ]:
# The @total_ordering decorator from the functools module is a powerful tool.
# If you define __eq__ and one other rich comparison method (like __lt__), 
# it will automatically generate the implementations for all the others (<=, >, >=).
@total_ordering
class Vector(Sequence):
    """A simple, immutable Vector class demonstrating key data model protocols.
    
    By inheriting from collections.abc.Sequence, we are formally declaring that
    this class behaves like a sequence. This is a form of "semantic typing."
    The ABC will ensure at class creation time that we have implemented
    the required methods (__len__ and __getitem__). In return, it provides default
    implementations for other sequence methods like __iter__, __reversed__, 
    index(), and count(), saving us boilerplate code and ensuring standard behavior.
    """
    
    def __init__(self, components):
        # We store components in an immutable tuple. This is a critical design choice.
        # For an object to be hashable (and thus usable as a dictionary key or set element),
        # its hash value must be constant over its lifetime. Making the object immutable
        # by storing its state in a tuple is the standard way to ensure this.
        self._components = tuple(float(x) for x in components)
        
    # --- Protocol 1: Object Representation ---
    def __repr__(self) -> str:
        """Provides the official, unambiguous string representation for developers.
           By convention, this should be a string of code that can recreate the object:
           eval(repr(v)) == v should hold true."""
        return f"Vector({list(self._components)})"
    
    def __str__(self) -> str:
        """Provides the user-friendly string representation for print() and str()."""
        return f"<{', '.join(f'{c:.2f}' for c in self._components)}>"
        
    # --- Protocol 2: Equality, Hashing, and Ordering ---
    def __eq__(self, other) -> bool:
        """Defines value equality (==) for two Vector objects."""
        if not isinstance(other, Vector):
            # NotImplemented is a special singleton that signals to the runtime that
            # this operation is not supported for the given operand types. It allows
            # Python to then try the reflected operation on the other object 
            # (e.g., if we wrote `v1 == 'not_a_vector'`, Python would then try 
            # `'not_a_vector'.__eq__(v1)`). This is more robust than returning False.
            return NotImplemented
        return self._components == other._components
    
    def __hash__(self) -> int:
        """Makes Vectors hashable, allowing them to be used in sets and dict keys.
           The hash is based on the immutable _components tuple, upholding the rule
           that if a == b, then hash(a) == hash(b)."""
        return hash(self._components)
    
    def __lt__(self, other) -> bool:
        """Implements less-than comparison based on vector magnitude."""
        if not isinstance(other, Vector):
            return NotImplemented
        return abs(self) < abs(other)

    # --- Protocol 3: Container Emulation ---
    def __len__(self) -> int:
        """Allows the built-in len() function to work on our Vector."""
        return len(self._components)
    
    def __getitem__(self, index):
        """Enables indexing (e.g., v[0]) and slicing (e.g., v[1:3])."""
        if isinstance(index, slice):
            # If the user provides a slice, we return a new Vector with the sliced components.
            return Vector(self._components[index])
        # Otherwise, we return the individual component.
        return self._components[index]
        
    # --- Protocol 4: Numeric Emulation ---
    def __abs__(self) -> float:
        """Calculates the Euclidean norm (L2 norm) of the vector for abs()."""
        return math.sqrt(sum(x * x for x in self))
    
    def __add__(self, other):
        """Implements vector addition using the `+` operator."""
        if not isinstance(other, Vector) or len(self) != len(other):
            return NotImplemented 
        components = (a + b for a, b in zip(self, other))
        return Vector(components)
        
    def __mul__(self, scalar):
        """Implements scalar multiplication (e.g., `v * 3`)."""
        if not isinstance(scalar, (int, float)):
            return NotImplemented
        return Vector(c * scalar for c in self)
        
    def __rmul__(self, scalar):
        """Implements reflected scalar multiplication (e.g., `3 * v`).
           This is called when `scalar * self` is evaluated, because the float `scalar`
           doesn't know how to multiply itself by a Vector. It effectively delegates
           the operation back to our class's `__mul__` method."""
        return self * scalar
        
    def __matmul__(self, other):
        """Implements the dot product using the `@` matrix multiplication operator (PEP 465)."""
        if not isinstance(other, Vector) or len(self) != len(other):
            return NotImplemented
        return sum(a * b for a, b in zip(self, other))
    
    # --- Protocol 5: Boolean Content (Truthiness) ---
    def __bool__(self) -> bool:
        """Defines the Vector's truth value. A Vector is False only if its magnitude is zero."""
        return abs(self) != 0

In [ ]:
sec("Vector Class Demonstration")
v1 = Vector([1, 2, 3]); v2 = Vector([4, 5, 6]); v3 = Vector([1, 2, 3])

note("Protocol 1: Representation")
print(f"  Developer view (__repr__): {repr(v1)}")
print(f"  User view (__str__):      {str(v1)}")
print(f"  The repr is executable: eval(repr(v1)) == v1 is {eval(repr(v1)) == v1}")

note("Protocol 2: Equality, Hashing, and Ordering")
print(f"  Identity (v1 is v3): {v1 is v3} (Different objects in memory)")
print(f"  Equality (v1 == v3): {v1 == v3} (Same value, due to __eq__)")
value_map = {v1: 'Policy Vector A'}
print(f"  Can v3 look up v1 in a dict? Yes: '{value_map[v3]}' (due to __hash__ and __eq__)")
print(f"  Ordering (v1 < v2):  {v1 < v2} (True, because abs(v1) < abs(v2))")
print(f"  Ordering (v1 >= v2): {v1 >= v2} (False, thanks to @total_ordering)")

note("Protocol 3: Container Emulation (as a Sequence)")
print(f"  Length: len(v1) -> {len(v1)}")
print(f"  Indexing: v1[0] -> {v1[0]}")
print(f"  Slicing: v1[1:] -> {v1[1:]} (Now returns a Vector!)")
print(f"  Membership (3.0 in v1): {3.0 in v1} (Provided by Sequence ABC)")
print("  Iteration: [x for x in v1] ->", [x for x in v1])
print(f"  Count (v1.count(1.0)): {v1.count(1.0)} (Provided by Sequence ABC)")

note("Protocol 4: Numeric Emulation")
print(f"  Addition (v1 + v2): {v1 + v2}")
print(f"  Scalar multiplication (v1 * 3): {v1 * 3}")
print(f"  Reflected multiplication (3 * v1): {3 * v1}")
print(f"  Dot product (v1 @ v2): {v1 @ v2:.2f}")
print(f"  Magnitude (abs(v2)): {abs(v2):.2f}")

note("Protocol 5: Boolean Content")
zero_vector = Vector([0, 0, 0])
print(f"  if v1: ... -> {'Truthy' if v1 else 'Falsy'}")
print(f"  if zero_vector: ... -> {'Truthy' if zero_vector else 'Falsy'}")

### 2. The Context Manager Protocol: `with` Statements for Safe Resource Management

The `with` statement in Python provides a mechanism for robustly managing resources like files, network connections, or database sessions. It guarantees that resources are properly acquired before a block of code is executed and, critically, that they are always released afterward, even if errors occur within the block. It is a vast improvement over manual `try...finally` blocks.

This is enabled by the **context manager protocol**, which involves two special methods:
- `__enter__(self)`: Called when entering the `with` block. Its return value is optionally bound to the variable specified in the `as` clause.
- `__exit__(self, exc_type, exc_value, traceback)`: Called when exiting the `with` block. If an exception occurred, the details are passed as arguments. This method can inspect the exception and decide whether to suppress it (by returning `True`) or let it propagate (by returning `False` or `None`).

#### 2.1 Code Lab: A Timer Context Manager Class
We can create a simple `Timer` class that uses the context manager protocol to measure the execution time of a code block. This is a common and practical application relevant to profiling the performance of economic simulations or data processing tasks.

In [ ]:
class Timer:
    """A context manager for measuring the execution time of a code block."""
    def __enter__(self):
        self.start_time = time.perf_counter()
        self.elapsed_time = None
        # This object is what gets assigned to `t` in `with Timer() as t:`
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"[Timer] Block executed in: {self.elapsed_time:.4f} seconds")
        # By returning a 'falsy' value (like None or False), we ensure that if an
        # exception occurred inside the `with` block, it will be re-raised.
        return False

sec("Context Manager Protocol Demonstration")
note("Using the Timer to measure a computation:")
with Timer() as t:
    # Simulate a computationally intensive task
    total = sum(i**2 for i in range(1_000_000))
    print(f"Inside the 'with' block, t.elapsed_time is {t.elapsed_time}")

note(f"After the 'with' block, we can still access the timer's result: {t.elapsed_time:.4f}s")

note("Demonstrating that exceptions are not suppressed:")
try:
    with Timer():
        print("About to raise an error...")
        raise ValueError("This is a test error")
except ValueError as e:
    print(f"Successfully caught error outside the 'with' block: {e}")

#### 2.2 A More Pythonic Way: The `@contextmanager` Decorator

For simple context managers that just wrap a `try...finally` block, creating a full class can be verbose. The `contextlib` module provides the `@contextmanager` decorator to create a context manager from a simple generator function. This is a more concise and often more readable pattern.

The function must:
1.  Contain exactly one `yield` statement.
2.  Code before the `yield` is treated as the `__enter__` method.
3.  The value yielded is what's bound to the `as` variable.
4.  Code after the `yield` (typically in a `finally` block) is treated as the `__exit__` method.

In [ ]:
@contextmanager
def temporary_model_parameter(model, param_name, temp_value):
    """A context manager to temporarily change a model parameter for sensitivity analysis."""
    original_value = getattr(model, param_name)
    print(f"--> Entering context: Setting '{param_name}' from {original_value} to {temp_value}")
    setattr(model, param_name, temp_value)
    try:
        yield
    finally:
        print(f"<-- Exiting context: Restoring '{param_name}' to {original_value}")
        setattr(model, param_name, original_value)

# A simple dummy model class
class MyModel:
    def __init__(self, risk_aversion=2.0):
        self.risk_aversion = risk_aversion
    def run_simulation(self):
        print(f"    ... Running simulation with risk aversion = {self.risk_aversion}")

model = MyModel()
note("Running baseline simulation:")
model.run_simulation()

note("Running sensitivity analysis with a temporary parameter change:")
with temporary_model_parameter(model, 'risk_aversion', 5.0):
    model.run_simulation()

note("Parameter has been restored after the block:")
model.run_simulation()

### 3. The Descriptor Protocol: Managed Attribute Access for Robust Models

The **descriptor protocol** is one of Python's most powerful, yet often hidden, features. It is the core mechanism that underpins properties (`@property`), methods, class methods (`@classmethod`), and static methods (`@staticmethod`). A descriptor is an object attribute whose access behavior is controlled by methods in the descriptor's class. It allows you to intercept attribute access and run custom logic.

A class becomes a descriptor if it implements any of these methods:
- `__get__(self, instance, owner)`: Controls how the attribute's value is retrieved.
- `__set__(self, instance, value)`: Controls how the attribute's value is set or changed.
- `__delete__(self, instance)`: Controls how the attribute is deleted.

A descriptor is assigned as a *class attribute*. When you then access that attribute on an *instance* of the class, Python's lookup mechanism finds the descriptor on the class and invokes the appropriate special method, as illustrated below. This interception is a powerful pattern for creating self-validating data structures, a common need in economic modeling where parameters must often conform to certain constraints (e.g., a savings rate must be between 0 and 1). By encapsulating validation logic in a descriptor, you create a reusable component that can be applied across many different models and attributes. This is superior to validation in `__init__` because it protects the attribute's integrity throughout the object's entire lifecycle, not just at creation.

![The Descriptor Protocol Lookup Chain](../images/1.4-descriptor-protocol.png)

#### Code Lab: A Self-Validating Economic Model
We will build a `PositiveValue` descriptor to ensure that certain economic parameters (e.g., capital stock, savings rate) are always positive. This creates a reusable component for building robust models where parameters are guaranteed to be economically sensible upon assignment.

In [ ]:
class PositiveValue:
    """A data descriptor that enforces positivity for a numeric attribute."""
    
    def __set_name__(self, owner_class, name):
        # This is called when the descriptor is created. We store the public
        # attribute's name (e.g., 'capital') to use for storage and in error messages.
        # This makes the descriptor reusable for any attribute name.
        self.public_name = name
        self.private_name = '_' + name # e.g., '_capital'

    def __get__(self, instance, owner_class):
        # `instance` is the object the descriptor is being accessed from (e.g., `params`)
        if instance is None:
            return self # Accessing from the class, e.g., SolowModel.capital_share
        # Retrieve the value from the instance's __dict__ using the private name
        return getattr(instance, self.private_name, None)

    def __set__(self, instance, value):
        # This method is called whenever someone tries to assign a value, e.g., `params.capital_share = 0.4`
        if not isinstance(value, (int, float)):
            raise TypeError(f"'{self.public_name}' must be a number.")
        if value <= 0:
            raise ValueError(f"'{self.public_name}' must be positive.")
        # If validation passes, the value is stored in the instance's __dict__ with the private name.
        setattr(instance, self.private_name, value)

class SolowModelParameters:
    """An example class using descriptors for parameter validation."""
    # Assigning the descriptor instance to a class attribute. This is where the magic happens.
    capital_share = PositiveValue()
    savings_rate = PositiveValue()

    def __init__(self, capital_share, savings_rate):
        # The assignment here triggers the descriptor's __set__ method for each parameter.
        self.capital_share = capital_share
        self.savings_rate = savings_rate

sec("Descriptor Protocol for Data Validation")
note("Attempting to create a valid model...")
try:
    params = SolowModelParameters(capital_share=0.33, savings_rate=0.2)
    print(f"-> Model created successfully. Alpha: {params.capital_share}, Savings: {params.savings_rate}")
except (ValueError, TypeError) as e:
    print(f"Caught unexpected error: {e}")

note("Attempting to assign an invalid negative value to savings_rate...")
try:
    params.savings_rate = -0.1
except ValueError as e:
    print(f"-> Caught expected error: {e}")

note("Attempting to assign an invalid non-numeric value...")
try:
    params.capital_share = "one-third"
except TypeError as e:
    print(f"-> Caught expected error: {e}")

### 4. Dynamic Attribute Access: `__getattr__` for Lazy Loading

Python provides a hook for intercepting access to attributes that do not otherwise exist in an object.

`__getattr__(self, name)` is a **fallback method**. It is only called when a requested attribute `name` is **not found** through the standard mechanisms (i.e., not in the instance's `__dict__`, its class, or any parent classes). This makes it ideal for patterns like lazy loading or creating proxy objects, where you want to compute or fetch a value only when it is first requested.

> **Warning: `__getattr__` vs. `__getattribute__`**
> Do not confuse `__getattr__` with `__getattribute__`. The latter is called for *every* attribute access, regardless of whether it exists or not. Overriding `__getattribute__` is difficult to do correctly without causing infinite recursion (e.g., by accessing `self.name` inside `__getattribute__`, which then calls `__getattribute__` again). It is a much more powerful tool but should be avoided unless absolutely necessary. For almost all use cases, `__getattr__` is the safer and more appropriate choice.

#### Code Lab: Lazy Loading of Datasets

Imagine a class that represents a large collection of datasets stored on disk. Loading all the data into memory when the object is created would be inefficient and slow. We can use `__getattr__` to implement lazy loading, where a specific dataset is only read from the disk the first time it is accessed.

In [ ]:
class LazyDataSource:
    """A class that only loads data from a source when an attribute is first accessed."""
    
    def __init__(self, source_path):
        self._source_path = source_path
        # In a real scenario, this might be a path to a directory of CSVs or a database connection.
        self._available_series = {'GDP', 'CPI', 'UNEMPLOYMENT'} # Assume we know what's available
        print(f"(LazyDataSource for '{source_path}' initialized. No data loaded yet.)")
        
    def __getattr__(self, name):
        # This method is only called if `name` is NOT found in self.__dict__.
        print(f"--> __getattr__ triggered for '{name}'. Trying to load...")
        
        if name.upper() in self._available_series:
            # Simulate loading data from a file
            print(f"--> Reading '{name.upper()}.csv' from disk...")
            # Create some dummy data
            value = pd.Series(np.random.rand(10) * 100, name=name.upper())
            
            # CRUCIAL STEP: Cache the loaded value in the instance's __dict__.
            # The next time this attribute is accessed, it will be found directly,
            # and __getattr__ will NOT be called again for it.
            setattr(self, name, value)
            return value
        else:
            # If the attribute is truly not available, we must raise AttributeError.
            raise AttributeError(f"'{type(self).__name__}' has no data series named '{name}'")

sec("Lazy Loading with __getattr__")
data_repo = LazyDataSource("/path/to/data")

note("First access to 'GDP':")
print(f"  GDP data (first 3 obs):\n{data_repo.GDP.head(3)}")

note("Second access to 'GDP' (should be cached, no loading message expected):")
print(f"  GDP data (first 3 obs):\n{data_repo.GDP.head(3)}")

note("Accessing a different dataset, 'CPI':")
print(f"  CPI data (first 3 obs):\n{data_repo.CPI.head(3)}")

note("Accessing a non-existent dataset:")
try:
    _ = data_repo.M2_SUPPLY
except AttributeError as e:
    print(f"-> Caught expected error: {e}")

### 5. Advanced Protocols for Framework Design

For economists building reusable modeling libraries, Python offers even more advanced tools for defining clear, robust, and extensible APIs. Abstract Base Classes and Metaclasses allow a library author to enforce a consistent structure across a wide range of user-defined models.

#### 5.1 Abstract Base Classes (ABCs): Defining an Interface

An **Abstract Base Class (ABC)** defines a common interface for a set of subclasses. It allows you to specify certain methods that any concrete (i.e., non-abstract) subclass *must* implement. This is a powerful tool for creating frameworks. For example, you could define a `GenericModel` ABC that requires all economic models inheriting from it to have a `.solve()` and a `.simulate()` method.

This is done using the `abc` module. You inherit from `abc.ABC` and use the `@abstractmethod` decorator.

In [ ]:
class GenericModel(ABC):
    """An Abstract Base Class for any economic model in our framework."""
    
    def __init__(self, params):
        self.params = params
        self.solution = None
        
    @abstractmethod
    def solve(self):
        """Solve the model. This must be implemented by all subclasses."""
        pass
        
    @abstractmethod
    def simulate(self, n_periods):
        """Simulate the model for a number of periods. Must be implemented."""
        pass

    def summary(self):
        return f"Model of type {type(self).__name__} with parameters {self.params}."

sec("Abstract Base Class Demonstration")
note("Trying to instantiate the ABC directly will fail:")
try:
    m = GenericModel({})
except TypeError as e:
    print(f"-> Caught expected error: {e}")

note("Now, let's create a concrete implementation:")
class SimpleGrowthModel(GenericModel):
    def solve(self):
        alpha = self.params['alpha']
        beta = self.params['beta']
        # In a real model, this would be a complex calculation
        self.solution = {'steady_state_k': (alpha / (1/beta - 1))**(1/(1-alpha))}
        print("SimpleGrowthModel solved.")
        return self.solution
        
    def simulate(self, n_periods):
        print(f"Simulating for {n_periods} periods...")
        return np.random.rand(n_periods)

growth_params = {'alpha': 0.33, 'beta': 0.95}
g_model = SimpleGrowthModel(growth_params)
g_model.solve()
print(f"Model summary: {g_model.summary()}")

#### 5.2 Metaclasses: The Classes of Classes

Metaclasses are a deep and powerful feature of Python. Just as a class defines the behavior of its instances, a **metaclass** defines the behavior of its classes. `type` is the default metaclass for all classes in Python.

By writing your own metaclass, you can intercept the creation of a class object itself and modify it. This allows you to enforce constraints or add features to classes automatically, without requiring decorators or inheritance. While often overkill, it's a key tool for advanced library and framework design.

For example, a metaclass could automatically register every new model class in a central registry, or ensure that all model classes have a `DESCRIPTION` attribute.

In [ ]:
MODEL_REGISTRY = {}

class ModelRegistryMeta(type):
    """A metaclass that automatically registers new model classes."""
    def __new__(cls, name, bases, dct):
        # `cls` is the metaclass itself (ModelRegistryMeta)
        # `name` is the name of the class being created (e.g., 'KeynesianModel')
        # `bases` is a tuple of parent classes
        # `dct` is the dictionary of the class's attributes and methods
        
        new_class = super().__new__(cls, name, bases, dct)
        if name != 'RegisteredModel': # Don't register the base class
            print(f"[Metaclass] Registering model: {name}")
            MODEL_REGISTRY[name] = new_class
        return new_class

# Any class that uses this metaclass will be processed by it at creation time.
class RegisteredModel(metaclass=ModelRegistryMeta):
    pass

sec("Metaclass Demonstration")
note("Defining two new models that inherit from RegisteredModel...")

class KeynesianModel(RegisteredModel):
    pass

class RBCModel(RegisteredModel):
    pass

note("The model registry now contains the new classes, added automatically:")
print(MODEL_REGISTRY)

### 6. Performance Optimization with `__slots__`

By default, Python stores an object's attributes in a special dictionary called `__dict__`. This makes Python very flexible, as you can add new attributes to an instance at any time. However, dictionaries have a significant memory overhead.

In performance-critical applications where you might create millions of small objects (e.g., in an agent-based model), this memory usage can become a bottleneck. The `__slots__` attribute provides a solution. By defining `__slots__` in a class, you are telling Python *not* to use a `__dict__` and to only allocate space for a fixed set of attributes.

**Benefits:**
- **Massively reduced memory footprint.**
- **Faster attribute access.**

**Drawbacks:**
- You cannot add new attributes to an instance that are not listed in `__slots__`.
- The class will not have a `__dict__` attribute.

In [ ]:
class AgentWithoutSlots:
    def __init__(self, wealth, age):
        self.wealth = wealth
        self.age = age

class AgentWithSlots:
    __slots__ = ['wealth', 'age'] # Define the fixed set of attributes
    def __init__(self, wealth, age):
        self.wealth = wealth
        self.age = age

sec("Memory Usage with and without __slots__")
a1 = AgentWithoutSlots(100.0, 40)
a2 = AgentWithSlots(100.0, 40)

note(f"Memory usage of standard object (with __dict__): {sys.getsizeof(a1) + sys.getsizeof(a1.__dict__)} bytes")
note(f"Memory usage of slotted object: {sys.getsizeof(a2)} bytes")

note("Attempting to add a new attribute:")
a1.is_employed = True # This works
print("-> Can add attribute to standard object.")
try:
    a2.is_employed = True # This will fail
except AttributeError as e:
    print(f"-> Caught expected error for slotted object: {e}")

### 7. Exercises

1.  **The Hash-Equality Invariant**
    The fundamental rule of hashing is: if `a == b`, it must be true that `hash(a) == hash(b)`. This requires that an object's hash value remains constant throughout its life. 
    - **Task:** Create a simple `MutablePoint` class that stores its `x` and `y` coordinates in a `list`. Implement `__eq__` and `__hash__` based on the list's contents. Create an instance `p1 = MutablePoint([1, 2])`. Add `p1` to a set. Then, mutate `p1` by changing its list in-place (e.g., `p1.coords[0] = 10`). 
    - **Analysis:** Show that `p1` is now in a corrupted state within the set (i.e., you can't retrieve it, and `p1 in my_set` may give inconsistent results). Explain why this demonstrates that only immutable objects should be hashable.

2.  **Advanced Descriptor: `BoundedParameter`**
    - **Task:** Create a descriptor class `BoundedParameter`. Its constructor should accept `min_value` and `max_value`. The descriptor's `__set__` method should enforce that any value assigned to the attribute falls within this inclusive range, raising a `ValueError` otherwise.
    - **Application:** Demonstrate its use on a `GrowthModel` class with attributes like `beta = BoundedParameter(0.9, 0.99)` and `alpha = BoundedParameter(0.1, 0.5)` to ensure model parameters are always economically sensible.

3.  **Context Manager with Decorator**
    - **Task:** Re-implement the `SeededRun` context manager from the previous exercise, but this time use the `@contextmanager` decorator from the `contextlib` module instead of a class.
    - **Hint:** The function should `yield` after setting the new seed, and the `finally` block should restore the old state.

4.  **Abstract Base Class for Solvers**
    - **Task:** Create an ABC named `NumericalSolver` with an abstract method `solve(initial_guess)`. Then, create two concrete subclasses, `NewtonSolver` and `BisectionSolver`, that inherit from `NumericalSolver` and provide their own implementations of the `solve` method (they can just be print statements for this exercise). 
    - **Analysis:** Explain why this structure would be beneficial for a library that offers multiple ways to solve the same economic model.

### 8. Curated References and Further Reading

**Definitive Texts on the Data Model**
- Ramalho, L. (2022). *Fluent Python*, 2nd Edition. O'Reilly Media. (The canonical, in-depth guide to the Python data model and creating Pythonic classes. Chapters 1, 10, 13, and 23 are particularly relevant.)
- Beazley, D. M., & Jones, B. K. (2013). *Python Cookbook*, 3rd Edition. O'Reilly Media. (Chapter 8 provides many practical recipes demonstrating data model features for advanced class design.)

**Key Talks and Primary Sources**
- Python Documentation: [The official data model reference](https://docs.python.org/3/reference/datamodel.html). (The ultimate source of truth, though dense.)
- Hettinger, R. (2013). *Python's Class Development Toolkit*. PyCon US 2013. [Watch on YouTube](https://www.youtube.com/watch?v=HTLu2DFOdTg). (An excellent and influential talk on the practical application of these protocols for building robust APIs.)
- Beazley, D. (2012). *Python 3 Metaprogramming*. PyCon US 2012. [Watch on YouTube](https://www.youtube.com/watch?v=sPiWg5jSoZI). (A classic, mind-bending talk on the power of descriptors and metaclasses.)